<a href="https://www.kaggle.com/code/mohamedabuelnasr/digit-recognizer?scriptVersionId=129070871" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [2]:
!pip install numpy==1.22.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 53.5 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.5
    Uninstalling numpy-1.23.5:
      Successfully uninstalled numpy-1.23.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.11.0 requires protobuf<3.20,>=3.9.2, but you have protobuf 3.20.3 which is incompatible.
tensorflow-serving-api 2.11.0 requires protobuf<3.20,>=3.9.2, but you have protobuf 3.20.3 which is incompatible.
pymc3 3.11.5 requires scipy<1.8.0,>=1.7.3, but you have scipy 1.9.3 which is incompatible.
librosa 0.10.0.post2 requires numpy!=1.22.0,!=1.22.1,!=1.22.2,>=1.20.3, but you have numpy 1.22.0 which is incompatible.
librosa 0.10.0.post2 requires soundfile>=0.12.1, but you have soundfile 0.11.0 which is incompatible.
beatrix-jupyterlab 2023.46.184821 requires jupyter-se

In [3]:
import numpy as np 
import pandas as pd 
import tensorflow as tf

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# Explore the data



In [4]:
data = pd.read_csv('/kaggle/input/digit-recognizer/train.csv')

In [5]:
data.shape

(42000, 785)

In [6]:
data.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
data.label.describe()

count    42000.000000
mean         4.456643
std          2.887730
min          0.000000
25%          2.000000
50%          4.000000
75%          7.000000
max          9.000000
Name: label, dtype: float64

In [8]:
data.iloc[0,1:].describe()

count    784.000000
mean      21.235969
std       64.660241
min        0.000000
25%        0.000000
50%        0.000000
75%        0.000000
max      255.000000
Name: 0, dtype: float64

# Sepration of data to train and validation
**95%** Train
**05%** Validation
<br/>
**labels** ranges from 0 -> 9
<br/>
**pixels** ranges from 0 -> 255. we need to normalize pixels to range from 0 -> 1
<br/>


In [11]:
data_count = tf.convert_to_tensor(data.groupby("label").count().iloc[:,0])
print(data_count)
print(tf.cast(data_count, dtype=tf.float32)*0.05)

tf.Tensor([4132 4684 4177 4351 4072 3795 4137 4401 4063 4188], shape=(10,), dtype=int64)
tf.Tensor(
[206.6     234.2     208.85    217.55    203.6     189.75    206.85
 220.05    203.15001 209.40001], shape=(10,), dtype=float32)


In [12]:
train_dataframes = []
validation_dataframes = []

for i in range(10):
    mask = np.random.rand(len(data.loc[data.label == i])) <= 0.95
    temp_train = data.loc[data.label == i][mask]
    temp_validation = data.loc[data.label == i][~mask]
    train_dataframes.append(temp_train)
    validation_dataframes.append(temp_validation)
    
for i in range(10):
    assert train_dataframes[i].iloc[:,0].count() + validation_dataframes[i].iloc[:,0].count() == data_count[i], "counts doesn't match"
    
train_data = pd.concat(train_dataframes)
validation_data = pd.concat(validation_dataframes)

In [13]:
train_data

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
17,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
23,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
54,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41962,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41969,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41975,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41992,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Shuffle the training data

In [15]:
train_data = train_data.sample(frac=1)
train_data

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
30052,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2900,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
20147,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
17250,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
14822,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16016,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5953,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
22269,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
20100,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Seprate the data into features and labels

In [17]:
train_data_features = train_data.copy()
train_data_labels = train_data_features.pop("label")

validation_data_features = validation_data.copy()
validation_data_labels = validation_data_features.pop("label")

In [18]:
validation_data_features

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
538,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
542,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
675,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
722,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40964,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41695,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41744,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41753,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
train_data_labels

30052    7
2900     3
20147    6
17250    0
14822    4
        ..
16016    4
5953     6
22269    7
20100    7
9390     7
Name: label, Length: 39957, dtype: int64

## convert pandas dataframes to tensorflow data sets

In [33]:
tf_train_X = tf.convert_to_tensor(train_data_features)
# convert 1D into 2D for the convolution
tf_train_X = tf.reshape(tf_train_X,[-1,28,28])
# normalize pixels from 0->255 into 0 -> 1
tf_train_X /= 255
tf_train_X = tf.expand_dims(tf_train_X, axis=-1)

tf_train_Y = tf.convert_to_tensor(train_data_labels)
tf_train_Y = tf_train_Y[:,tf.newaxis]

tf_validation_X = tf.convert_to_tensor(validation_data_features)
tf_validation_X = tf.reshape(tf_validation_X,[-1,28,28])
tf_validation_X /= 255

tf_validation_Y = tf.convert_to_tensor(validation_data_labels)
tf_validation_Y = tf_validation_Y[:,tf.newaxis]

print("Train data shape: ", tf_train_X.shape)
print("Train labels shape: ", tf_train_Y.shape)
assert tf_train_Y.shape[0] == tf_train_X.shape[0], "train data sizes doesn't match"

print("validation data shape: ", tf_validation_X.shape)
print("validation labels shape: ", tf_validation_Y.shape)
assert tf_validation_X.shape[0] == tf_validation_Y.shape[0], "validation data sizes doesn't match"

Train data shape:  (39957, 28, 28, 1)
Train labels shape:  (39957, 1)
validation data shape:  (2043, 28, 28)
validation labels shape:  (2043, 1)


In [34]:
train_dataset = tf.data.Dataset.from_tensor_slices((tf_train_X, tf_train_Y))
validation_dataset = tf.data.Dataset.from_tensor_slices((tf_validation_X, tf_validation_Y))

In [35]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
train_dataset = train_dataset.batch(512)
train_dataset = train_dataset.prefetch(AUTOTUNE)

validation_dataset = validation_dataset.batch(264)
validation_dataset = validation_dataset.prefetch(AUTOTUNE)

# Building the model

In [36]:
model = tf.keras.Sequential([
    tf.keras.Input((28, 28,1)),
    tf.keras.layers.Conv2D(filters=64, kernel_size=3, strides=1, activation='relu', name="conv2"),
    tf.keras.layers.MaxPool2D(pool_size=2),
    tf.keras.layers.Conv2D(filters=64, kernel_size=3, strides=1, activation='relu', name="conv3"),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=128, activation="relu", name="dense1"),
    tf.keras.layers.Dense(units=10, activation="linear", name="dense2")
])
model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(lr=0.001),
    metrics=["accuracy"]
)
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2 (Conv2D)              (None, 26, 26, 64)        640       
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 13, 13, 64)       0         
 2D)                                                             
                                                                 
 conv3 (Conv2D)              (None, 11, 11, 64)        36928     
                                                                 
 flatten_1 (Flatten)         (None, 7744)              0         
                                                                 
 dense1 (Dense)              (None, 128)               991360    
                                                                 
 dense2 (Dense)              (None, 10)                1290      
                                                      

In [38]:
model.fit(train_dataset, validation_data=validation_dataset, epochs=25)

Epoch 1/25
79/79 [==============================] - 33s 420ms/step - loss: 0.0336 - accuracy: 0.9903 - val_loss: 0.0406 - val_accuracy: 0.9873
Epoch 2/25
79/79 [==============================] - 33s 422ms/step - loss: 0.0244 - accuracy: 0.9931 - val_loss: 0.0450 - val_accuracy: 0.9843
Epoch 3/25
79/79 [==============================] - 33s 418ms/step - loss: 0.0201 - accuracy: 0.9941 - val_loss: 0.0392 - val_accuracy: 0.9858
Epoch 4/25
79/79 [==============================] - 33s 423ms/step - loss: 0.0158 - accuracy: 0.9954 - val_loss: 0.0427 - val_accuracy: 0.9868
Epoch 5/25
79/79 [==============================] - 33s 422ms/step - loss: 0.0133 - accuracy: 0.9964 - val_loss: 0.0453 - val_accuracy: 0.9853
Epoch 6/25
79/79 [==============================] - 33s 423ms/step - loss: 0.0115 - accuracy: 0.9971 - val_loss: 0.0514 - val_accuracy: 0.9853
Epoch 7/25
79/79 [==============================] - 34s 425ms/step - loss: 0.0111 - accuracy: 0.9968 - val_loss: 0.0489 - val_accuracy: 0.9868

In [39]:
model.evaluate(validation_dataset)

8/8 [==============================] - 0s 57ms/step - loss: 0.0746 - accuracy: 0.9848


[0.07463663816452026, 0.9848262071609497]

# Generate Predictions for test set

In [40]:
data_test = pd.read_csv('/kaggle/input/digit-recognizer/test.csv')

In [41]:
data_test

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [42]:
test_X = tf.convert_to_tensor(data_test)
# convert 1D into 2D for the convolution
test_X = tf.reshape(test_X,[-1,28,28])
# normalize pixels from 0->255 into 0 -> 1
test_X /= 255
test_X

<tf.Tensor: shape=(28000, 28, 28), dtype=float64, numpy=
array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0

In [43]:
test_dataset = tf.data.Dataset.from_tensor_slices(test_X)
test_dataset = test_dataset.batch(264)
test_dataset = test_dataset.prefetch(AUTOTUNE)

In [44]:
logits = model.predict(test_dataset)


107/107 [==============================] - 6s 57ms/step


In [45]:
logits.shape

(28000, 10)

In [46]:
predictions = tf.nn.softmax(logits, axis=1)

In [47]:
predictions = tf.argmax(predictions,axis=1)
predictions

<tf.Tensor: shape=(28000,), dtype=int64, numpy=array([2, 0, 9, ..., 3, 9, 2])>

In [48]:
ids  = tf.range(1, predictions.shape[0]+1)
submmision_dic = {
    "ImageId": ids,
    "Label": predictions
}
sub_df = pd.DataFrame(submmision_dic)
sub_df.to_csv('/kaggle/working/sub3.csv', index=False)

In [50]:
my_sub = pd.read_csv('/kaggle/working/sub3.csv')
my_sub

,ImageId,Label
0,1,2
1,2,0
2,3,9
3,4,9
4,5,3
...,...,...
27995,27996,9
27996,27997,7
27997,27998,3
27998,27999,9
